In [29]:
import numpy as np
import pandas as pd
import math 

params = {
    'pi' : [np.pi],
    'alpha' : [0.1],
    'rho' : [1.225],
    'lambda_opt' : [7.6],
    'Cp_max' : [0.486],
    'area' :  [math.pi * (57.5**2)],
    'a' : [3],
    'k' : [2],
    'h' : [81],
    'r0' : [1.5],
    'R' : [57.5],
    'zeta' : [0.6],
    'zeta2' : [0.45],
    'zeta3' : [0.9],
    'omegan' : [11.11],
    'omegan2' : [5.73],
    'omegan3' : [3.42],
    'Bdt' : [755.49],
    'Kdt' : [2.7*1e9],
    'Jr' : [55*1e6],
    'Jg' : [55*1e6],
}

df = pd.DataFrame(params,index=None)
df.to_csv('parameters.csv',index=False)

path = 'parameters.csv'
df = pd.read_csv(path)
params = df.to_dict()

In [ ]:
class Dynamics:
    def __init__(self, **params):

        self.pi = params['pi'] 
        self.alpha = params['alpha'] 
        self.rho = params['rho'] 
        self.lambda_opt = params['lambda_opt'] 
        self.Cp_max = params['Cp_max'] 
        self.area = params['area']
        self.a = params['a']
        self.k = params['k']
        self.h = params['h']
        self.r0 = params['r0']
        self.R = params['R']
        self.Kdt = params['Kdt']
        self.Bdt = params['Bdt']
        self.Jr = params['Jr']
        self.Jg = params['Jg']
        self.Bg = params['Bg']
        self.Ng = params['Ng']
        self.Ndt = params['Ndt']

        self.Tg_r = None
        self.Tg_r2 = None
        self.B_r = None
        self.tg = None
        self.xi = None
        self.Wn = None
        self.dTg = None
        self.dB = None
        self.ddB = None
        self.dO = None
        self.dWg = None
        self.dWr = None

        self.Tg = None
        self.B = None
        self.O = None
        self.Wg = None
        self.Wr = None

        self.X = np.array([self.Tg,self.B,self.dB,self.O,self.Wg,self.Wr])
        self.dX = np.array([self.dTg,self.dB,self.ddB,self.dO,self.dWg,self.dWr])


    def set_state(self, X):
        """
        Define the current state of the plant. 
        
        x : array-like
            Current state [omega_r, omega_g, theta]
        """
        self.X = X.flatten()

    def dynamics(self, t, X, U):

        Bdt = self.Bdt 
        Kdt = self.Kdt 
        Jr = self.Jr 
        Jg = self.Jg 
        Bg = self.Bg
        Ng = self.Ng
        Ndt = self.Ndt
        tg = self.tg
        xi = self.xi
        Wn = self.Wn
        one = np.array([[1]])

        A11 = (-1/tg)*one
        A54 = -(Wn**2)*one 
        A55 = -2*xi*Wn
        a71 = -(1/Jg) * one 
        a77 = -((Ndt*Bdt/(Jg*(Ng**2))) + (Bg/Jg))
        a84 = 
        a88 = 
        A0=np.array([[A11,    0,   0,           0,           0,           0],
                     [  0,    0,  one,           0,           0,           0],
                     [  0,  A54, A55,           0,           0,           0],
                     [  0,    0,   0,           0,        1/Ng,           1],
                     [a71,    0,   0, Kdt/(Jg*Ng),         a77, Bdt/(Ng*Jg)],
                     [   0, a84,   0,   -Kdt/(Jr), Bdt/(Ng*Jg),         a88]])
        
        B = np.array([[1/Jr,      0],
                      [    0, -1/Jg],
                      [    0,     0]])
        
        dX = (A@X.reshape(-1,1))+(B@U.reshape(-1,1))
        self.dX = dX.flatten()
        return dX.flatten()
    
    def compute_input(self, v, X, uk=0):
        """
        Compute control input u as u = Kx + uf, where
        uf is a feedforward term.
        
        Parameters:
        x: numpy array
            Current state [x, y, theta, vx, vy, w]
        K: numpu array matrix-alike
            Gain matrix.
        """
        
        wr = X[0]
        wg = X[1]
        rho = self.rho
        A = self.A
        R = self.R
        lmbd_star = self.lmbd_opt + uk
        lmbd = wr*R/v
        Cp_star = self.Cp_star
        if lmbd_star != self.lmbd_opt:
            Cp_star = Cp_calc(lmbd_star)
        Cp = Cp_calc(lmbd)
        self.cp_.append(Cp_star)
        self.lmbd_.append(lmbd_star)
        K_mppt = 0.5 * rho * A * (R**3) * Cp_star/(lmbd_star**3)
        #print(lmbd_star,Cp_star,K_mppt)
        tR = 0.5*rho*A*Cp*(v**3)/wr
        tG = K_mppt*(wg**2)
        du = np.array([tR,tG])
        self.U = du
        self.K_mppt = np.append(self.K_mppt,K_mppt)
        
        return du
    
    def iterate(self, u, dt):
        """
        Propagate the system forward in time by 'dt' using 
        numerical integration.
        
        Parameters:
        u : array-like
            Input vector [u1, u2]
        dt : float
            Time step
        """
        #DOP853
        sol = solve_ivp(self.dynamics, [0, dt], self.X, 
            args=(u.flatten(),), method="DOP853", t_eval=[dt])
        self.X = sol.y[:, -1]
        return self.X
    
    def compute_output(self):
        X = self.X.reshape(1,-1)
        dX = self.dX.reshape(1,-1)
        B = self.Bdt
        tau_r, tau_g = self.U
        w_r, w_g, _ = self.X
        PG = tau_g*w_g
        PD = B*((w_r-w_g)**2)

        #if self.k>=300:
        EG = self.EG[-1]+PG
        ED = self.ED[-1]+PD

        self.PD = np.append(self.PD,PD)
        self.PG = np.append(self.PG,PG) 
        self.ED = np.append(self.ED,ED)
        self.EG = np.append(self.EG,EG) 
        self.X_ = np.append(self.X_,X,axis=0)  
        self.dX_ = np.append(self.dX_,dX,axis=0)  
        if self.k == 1:
            self.PD = np.delete(self.PD,0)
            self.PG = np.delete(self.PG,0) 
            self.ED = np.delete(self.ED,0)
            self.EG = np.delete(self.EG,0) 

        self.k = self.k + 1